In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, f1_score, accuracy_score, multilabel_confusion_matrix, ConfusionMatrixDisplay, confusion_matrix
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from src.data_processor.phase_3.prob2.v1 import  Phase3Prob2FeatureProcessor
import numpy as np
import gc
from collections import Counter

In [ ]:
feat_to_name = {'feature1': 'dur', 'feature2': 'proto', 'feature3': 'service', 'feature4': 'state', 'feature5': 'spkts', 'feature6': 'dpkts', 'feature7': 'sbytes', 'feature8': 'dbytes', 'feature9': 'sttl', 'feature10': 'dttl', 'feature11': 'sload', 'feature12': 'dload', 'feature13': 'sloss', 'feature14': 'dloss', 'feature15': 'sinpkt', 'feature16': 'dinpkt', 'feature17': 'sjit', 'feature18': 'djit', 'feature19': 'swin', 'feature20': 'stcpb', 'feature21': 'dtcpb', 'feature22': 'dwin', 'feature23': 'tcprtt', 'feature24': 'synack', 'feature25': 'ackdat', 'feature26': 'smean', 'feature27': 'dmean', 'feature28': 'trans_depth', 'feature29': 'response_body_len', 'feature30': 'ct_srv_src', 'feature31': 'ct_state_ttl', 'feature32': 'ct_dst_ltm', 'feature33': 'ct_src_dport_ltm', 'feature34': 'ct_dst_sport_ltm', 'feature35': 'ct_dst_src_ltm', 'feature36': 'is_ftp_login', 'feature37': 'ct_ftp_cmd', 'feature38': 'ct_flw_http_mthd', 'feature39': 'ct_src_ltm', 'feature40': 'ct_srv_dst', 'feature41': 'is_sm_ips_ports', 'label': 'label'}
name_to_feat = {v:k for k, v in feat_to_name.items()}

In [5]:
df = pd.read_parquet("F:/Data/MLOPS_2023/data_phase-3/phase-3/prob-2/raw_train.parquet")
print(df.shape)
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)
print(df.shape)
df.drop_duplicates(df.columns[:-1], keep=False, inplace=True)
print(df.shape)

In [9]:
df2 = pd.read_csv("F:/Data/MLOPS_2023/UNSW_NB15_training-set.csv")
df3 = pd.read_csv("F:/Data/MLOPS_2023/UNSW_NB15_testing-set.csv")
df2 = pd.concat([df2, df3])
del df3
gc.collect()
df2['label'] = df2['attack_cat']
mapping = {
            "Normal": "Normal",
            "DoS":"Denial of Service",
            "Reconnaissance":"Information Gathering",
            "Analysis": "Information Gathering",
            "Exploits": "Exploits",
            "Shellcode": "Malware",
            "Worms": "Malware",
            "Backdoor":"Malware",
            "Generic": "Other",
            "Fuzzers": "Denial of Service"
            }
df2["label"] = df2["label"].map(mapping)
print(df2["label"].value_counts(dropna=False))
print(df2.shape)
df2.drop(columns=['id','rate','attack_cat'],inplace=True)
df2.drop_duplicates(inplace=True)
print(df2["label"].value_counts(dropna=False))
print(df2.shape)
df2.drop_duplicates(df2.columns[:-1], keep=False, inplace=True)
print(df2.shape)
df2.columns = [name_to_feat[c] for c in df2.columns]


Normal                   93000
Other                    58871
Exploits                 44525
Denial of Service        40599
Information Gathering    16664
Malware                   4014
Name: label, dtype: int64
(257673, 45)
Normal                   85722
Exploits                 27434
Denial of Service        24718
Information Gathering    10686
Other                     7599
Malware                   3507
Name: label, dtype: int64
(159666, 42)
(151246, 42)


In [11]:
df3 = pd.concat([df, df2])
df3.drop_duplicates(inplace=True)
print(df3.shape)
df3.reset_index(drop=True, inplace=True)

In [21]:
HyperParameters={'objective': 'multiclass',
                 'n_estimators':200,
                 'learning_rate':0.7,
                 'max_depth':32,
                 'colsample_bytree':0.8,
                 'subsample':0.8,
                 'reg_alpha':1.2,
                 'reg_lambda':10,
                 'random_state':42}
            
hyper_parameters = HyperParameters.copy()

In [22]:
TARGET = 'label'
processor = Phase3Prob2FeatureProcessor()
new_df = processor.transform(df3)
kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
FEATURES = processor.data_features['features']
categorical = processor.data_features['categorical_features']
models = []
scores = []
oofs = np.empty(df3.shape[0], dtype='object')
    X_train = new_df.iloc[train_idx]
    y_train = df3.iloc[train_idx][TARGET]
    X_valid = new_df.iloc[valid_idx]
    y_valid = df3.iloc[valid_idx][TARGET]


    print(X_train.shape, X_valid.shape)
    print(y_train.shape, y_valid.shape)
    model = LGBMClassifier(**hyper_parameters)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric=["logloss"],
              categorical_feature=categorical,
              
              verbose=50)
    # model = CatBoostClassifier(iterations=500, 
    #                            learning_rate=0.1, max_depth=8, 
    #                             random_state=42,eval_metric='Accuracy', thread_count=-1,
    #                             )
    # model.fit(X_train, y_train,
    #           cat_features=categorical,
    #           eval_set=[(X_train, y_train), (X_valid, y_valid)]
    #           )
    models.append(model)
    y_pred = model.predict(X_valid)
    print(y_pred)
    oofs[valid_idx] = y_pred

    print(f"fold {i} : {accuracy_score(y_valid, y_pred)}")
    print(classification_report(y_valid, y_pred))
    scores.append(accuracy_score(y_valid, y_pred))

print(np.mean(scores), np.std(scores))
print(classification_report(df3[TARGET], oofs))
print(accuracy_score(df3[TARGET], oofs))

(121291, 41) (30323, 41)
(121291,) (30323,)
[50]	training's multi_logloss: 0.21735	valid_1's multi_logloss: 0.33339
[100]	training's multi_logloss: 0.162366	valid_1's multi_logloss: 0.335468
[150]	training's multi_logloss: 0.12765	valid_1's multi_logloss: 0.340573
[200]	training's multi_logloss: 0.10338	valid_1's multi_logloss: 0.345302
['Normal' 'Exploits' 'Normal' ... 'Information Gathering'
 'Denial of Service' 'Exploits']
fold 0 : 0.8709230617023381
                       precision    recall  f1-score   support

    Denial of Service       0.69      0.64      0.66      4505
             Exploits       0.84      0.89      0.86      5109
Information Gathering       0.86      0.83      0.84      1778
              Malware       0.68      0.62      0.65       392
               Normal       0.92      0.93      0.93     17082
                Other       0.97      0.91      0.94      1457

             accuracy                           0.87     30323
            macro avg       0.83    

In [24]:
gmodel = LGBMClassifier(**hyper_parameters)
gmodel.fit(new_df, df3[TARGET],
           eval_set=[(new_df,df3[TARGET])],
           eval_metric=["logloss"],
           verbose=50)

with open(f'../checkpoints/phase-3/prob-2/v1.pkl','wb') as file:
    pickle.dump(gmodel, file)

[50]	training's multi_logloss: 0.233132
[100]	training's multi_logloss: 0.174968
[150]	training's multi_logloss: 0.139594
[200]	training's multi_logloss: 0.116658


In [25]:
y_pred = gmodel.predict(new_df)

In [26]:
print(classification_report(df3[TARGET],y_pred))

                       precision    recall  f1-score   support

    Denial of Service       0.92      0.87      0.89     22522
             Exploits       0.97      0.99      0.98     25542
Information Gathering       0.99      0.99      0.99      8891
              Malware       0.94      0.97      0.96      1961
               Normal       0.97      0.98      0.98     85410
                Other       0.99      1.00      0.99      7288

             accuracy                           0.97    151614
            macro avg       0.97      0.97      0.97    151614
         weighted avg       0.97      0.97      0.97    151614



In [18]:
feats = pd.DataFrame({'feature':gmodel.feature_name_, 'score':gmodel.feature_importances_}).sort_values('score',ascending=False)

In [19]:
feats['feature'].values[:15]

array(['feature26', 'feature7', 'feature1', 'feature11', 'feature21',
       'feature20', 'feature18', 'feature25', 'feature24', 'feature23'],
      dtype=object)